In [ ]:
"""
The purpose of this python script is to be able to implement ABCR berths to each terminus station in the line.

Unfortunately, I couldn't find any pattern all of the terminus platforms in each respective line so that the code could do it 
all automatically. However, it is not very tedious to find all of these terminus platforms using opentraintimes. Once you locate
a terminus you can go to the INPUTS section and put the required inputs for the following functions to work so that you can add
the berth to that given platform. You will have to do this for every terminus platform you find in the line.

After running all the functions, the original JSON file will have bnew ABCR berths signals included in each terminus stations
with links in between them. Moreover, all of the inputs can be obtained through the DRIVE which makes the process more efficient.

Once you open the file again, in order to have pretty printing press SHIFT+Alt+F in Visual Studio.

Finally, a recommendation is to save a coy of the JSON tracks file on another folder in case something doesn't work as planned
as these functions will rewrite the original JSON file.
"""

In [7]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [8]:
def silencing_links(path: "json",x1: "float",x2: "float",y1: "float",y2: "float"):
    """
    The purpose of this function will be to silence all of the links of the nodes/signals between x1 and x2 as you will later
    see that it is easier to create new links in between all of them once the ABCR berths are added. Therefore, this function
    adds a _D at the end of the id, source and target in each link.
    
    path : Path of the JSON file in the computer. Don't forget to add an r before it as seen in the examples below.
    
    x1 : The x coordinate most to the left of the given range for x coordinates, if terminus signal going to right, then x1
    is the same as the coordinate of the terminus signal.
    
    x2 : The x coordinate most to the right of the given range for x coordinates, if terminus signal going to left, then x2
    is the same as the coordinate of the terminus signal.
    
    y1 : The bottom y coordinate in the y range.
    
    y2 : The upper y coordinate in the y range.
    
    """
    with open(path) as data_file:    
        data = json.load(data_file) 
    #This open the JSON file and the following line creates a dataframe of the 'nodes' in the JSON file.
    df_nodes = pd.DataFrame(data['nodes'])
    #The df_nodes is tranformed into a zipped list for having easier operations.
    list_nodes = list(zip(df_nodes['id'],df_nodes['x'],df_nodes['y'],df_nodes['name'],df_nodes['direction']))
    #The following empty list_new_nodes will be a list which will only include nodes that are in the given x and y range
    #after the for loop is run.
    list_new_nodes = []
    for i in list_nodes:
        if  (x1 < i[1] < x2 + 2) and ( y1 - 2 < i[2] < y2 + 2):
            list_new_nodes.append(i)
    df_links = pd.DataFrame(data['links'])
    #This will be a dataframe of the links in the JSON file and list_links its respective zipped list.
    list_links = list(zip(df_links['id'],df_links['source'],df_links['target']))   
    #list_new_links will be the list of all the links associated with the nodes from list_new_nodes in the x-y ranges.
    list_new_links = []

    for i in list_new_nodes:
        for b in list_links:
            if i[0] == (b[1] or b[2]):
                #i[0] is the id of the node which can apear either in the source or target of the link. This leads to the
                #addition of repeated links as the same link will be added twice once i[0] is the source and later when
                #it is the target.
                list_new_links.append(b)
    list_new_links = list(set(list_new_links))
    #Use of set function to get rid of repeated links.
    list_updated_links = []
    #This function will contain the updated links with most of them as they were initially except the ones in the x-y range,
    #which will have a _D at the end.
    for i in list_links:
        x = False
        for b in list_new_links:
            if (i[1] == b[1]) and (i[2] == b[2]):
                #if the same links are in list_links and list_new_links, then the _D is added to these links and these new
                #links are added to list_updated_links.
                new_id = i[0] +"_D"
                new_source = i[1] + "_D"
                new_target = i[2] + "_D"
                new_link = (new_id,new_source,new_target)
                list_updated_links.append(new_link)
                x = True
        if x == False:
            #Otherwise, if the links don't appear is list_new_links then they are just added to list_updated_links.
            list_updated_links.append(i)

    Id = []
    source = []
    target = []
    for i in list_updated_links:
        Id.append(i[0])
        source.append(i[1])
        target.append(i[2])
    df_update_links = pd.DataFrame(columns= ['id','source','target'])
    df_update_links['id'] = Id
    df_update_links['source'] = source
    df_update_links['target'] = target
    #After creating a new dataframe, this is converted to json format and then loaded as a json string which replaces the 
    #map links in the original JSON file. The last two lines rewrite the JSON file and the links in the x-y range are now
    #silenced.
    df_update_links_json = df_update_links.to_json(orient = 'records')
    json_object = json.loads(df_update_links_json)
    data['links'] = json_object
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [9]:
def creating_abcr(path,plat_id,signal_id,direction):
    """
    The purpose of this function is to add ABCR berth signals in the nodes map in the JSON file which will either go to the
    right or left of the given terminus signal in the terminus platform.
    
    path : Path of the JSON file in the computer. Don't forget to add an r before it as seen in the examples below.
    
    plat_id : ID of the terminus platform to which you want to add the ABCR berths.
    
    signal_id : ID of the terminus signal in the terminus platform.
    
    direction : either 'r' or 'l' (or whatever). 'r' will mean that the ABCR starting from A will go to the right of the 
    terminus signal. Otherwise, it will go to the left.
    
    Finally, the code will rewrite the 'nodes' map in the JSON file including the new berths.
    
    """
    with open(path) as data_file:    
        data = json.load(data_file) 
    #This opens the JSON file and then df_nodes and df_platforms are the respective dataframes of the 'nodes' and 'platforms'
    #as well as their respective zipped list. 
    df_nodes = pd.DataFrame(data['nodes'])
    list_nodes = list(zip(df_nodes['id'],df_nodes['x'],df_nodes['y'],df_nodes['name'],df_nodes['direction']))
    df_platforms = pd.DataFrame(data['platforms'])
    list_platforms = list(zip(df_platforms['id'],df_platforms['x1'],df_platforms['x2'],df_platforms['y1'],df_platforms['y2']))
    signal = 0
    for i in list_nodes:
        if i[0] == signal_id:
            signal = i
    platform = 0
    for i in list_platforms:
        if i[0] == plat_id:
            platform = i
    #After this for loops we have the signal and platform with the same ID's as in the inputs.
    integers = "0123456789"
    new_name = ""
    for i in signal[3]:
        if i in integers:
            new_name += i
    #This for loop is used so that only the number part in the signal ID is included after the A/B/C/R of the berth.
    #e.g. SN: 356 -> A356,B536,C356,D356.
    dx = (abs(platform[2] - platform[1]))/4
    #dx is ths difference between x1 and x2 in the platforms divided by 4.
    list_berths = ["A","B","C","R"]
    for i in range(len(list_berths)):
        Id =  "v" + str(i+1) + "_" + signal[0]
        #The id's of each berth will have a vi_ as a prefix for the id of the terminus signal, where i is an integer from 1
        #to 4.
        if direction == "r":
            x = signal[1] + (i+1)*dx
            #the x coordinate of each berth will be to the right of the terminus signal.
        else:
            #the x coordinate of each berth will be to the left of the terminus signal.
            x = signal[1] - (i+1)*dx
        y = signal[2]
        #the berths will have the same y coordinates.
        name = list_berths[i] + new_name
        #The name of the berths will be their respective letter and then the number in the name the signal.
        direction_nodes = signal[4]
        berth = (Id,x,y,name,direction_nodes)
        list_nodes.append(berth)
        #Appends the berths to the original list_nodes.
    Id_nodes = []
    x = []
    y = []
    name = []
    direction_nodes = []
    for i in list_nodes:
        Id_nodes.append(i[0])
        x.append(i[1])
        y.append(i[2])
        name.append(i[3])
        direction_nodes.append(i[4])
    df_update_nodes = pd.DataFrame(columns= ['id','x','y','name','direction'])
    df_update_nodes['id'] = Id_nodes
    df_update_nodes['x'] = x
    df_update_nodes['y'] = y
    df_update_nodes['name'] = name
    df_update_nodes['direction'] = direction_nodes
    df_update_nodes_json = df_update_nodes.to_json(orient = 'records')
    json_object = json.loads(df_update_nodes_json)
    data['nodes'] = json_object
    #The previous lines just create a new dataframe will the new berths included and then rewrites the 'nodes' map in the 
    #original JSON file.
    with open(path, 'w') as json_file:
        json.dump(data, json_file)
        
        
        

In [10]:
def creating_links(path,x1,x2,y1,y2,link_id,direction):
    """
    The purpose of this function is to create links in between all the nodes/signals and berths in a given x-y range, the
    same x-y range given for the silencing_links() function.
    
    path : Path of the JSON file in the computer. Don't forget to add an r before it as seen in the examples below.
    
    x1 : The x coordinate most to the left of the given range for x coordinates, if terminus signal going to right, then x1
    is the same as the coordinate of the terminus signal.
    
    x2 : The x coordinate most to the right of the given range for x coordinates, if terminus signal going to left, then x2
    is the same as the coordinate of the terminus signal.
    
    y1 : The bottom y coordinate in the y range.
    
    y2 : The upper y coordinate in the y range.
    
    link_id : The id of any link connected to the terminus signal.
    
    direction : either 'r' or 'l' (or whatever). 'r' will mean that the ABCR starting from A will go to the right of the 
    terminus signal. Otherwise, it will go to the left. Consequently, all links will go from either right to left or viceversa.
    
    """
    with open(path) as data_file:    
        data = json.load(data_file)
    #This opens the JSON file and then df_nodes and df_links are the respective dataframes of the 'nodes' and 'links'
    #as well as their respective zipped list.
    df_nodes = pd.DataFrame(data['nodes'])
    list_nodes = list(zip(df_nodes['id'],df_nodes['x'],df_nodes['y'],df_nodes['name'],df_nodes['direction']))
    df_links = pd.DataFrame(data['links'])
    list_links = list(zip(df_links['id'],df_links['source'],df_links['target']))
    #list_new_nodes is the list that will contain all the nodes in the given x-y range.
    list_new_nodes = []
    for i in list_nodes:
        if  (x1 - 1 < i[1] < x2 + 2 ) and ( y1 - 2 < i[2] < y2 + 2):
            list_new_nodes.append(i)
    #As direction is going to the right, all nodes and berths in list_new_nodes are ordered from smallest to largest x coordinate
    #so that the first node is the one most to the left and the final one most to the right.
    if direction == "r":
        list_new_nodes = sorted(list_new_nodes, key=lambda tup: tup[1])
    #Otherwise, it will be sorted from the biggest (right) x coordinate to the smaless (left)
    else:
        list_new_nodes = sorted(list_new_nodes, key=lambda tup: tup[1], reverse=True)
    #This for loop will be the one creating the links, as it uses the indexes of the nodes in list_new_nodes, it was necessary to
    #sort them out in terms of their x coordinates as they were before randomly distributed in list_new_nodes which could lead
    #to the wrong links being created.
    for i in range(len(list_new_nodes)):
        if i == len(list_new_nodes) -1:
            break
        else: 
            Id = "v" + str(1+i) + "_" + link_id
            source = list_new_nodes[i][0]
            target = list_new_nodes[i+1][0]
            new_link = (Id,source,target)
            list_links.append(new_link)
    Id = []
    source = []
    target = []
    for i in list_links:
        Id.append(i[0])
        source.append(i[1])
        target.append(i[2])
    df_update_links = pd.DataFrame(columns= ['id','source','target'])
    df_update_links['id'] = Id
    df_update_links['source'] = source
    df_update_links['target'] = target
    df_update_links_json = df_update_links.to_json(orient = 'records')
    json_object = json.loads(df_update_links_json)
    data['links'] = json_object
    #The previous lines just create a new dataframe will the new berths included and then rewrites the 'links' map in the 
    #original JSON file.
    with open(path, 'w') as json_file:
        json.dump(data, json_file)
    

In [23]:
"""
                                                        INPUTS
                                                
"""

path = r'C:\Users\Gyalpo\Documents\Cogitare\drive\assets\data\ds\lner-tracks.json'
#Either the x1 or x2 will be the coordinate from the terminus signal. The other x coordinate should be an x coordinate a bit far
#from the end of the platform in the repsective direction.
x1 = -271416.8       
x2 = -268395.8 
y1 = 120606
y2= 120608
plat_id = "HTRWTM5_P4"
signal_id = "71_gw180"
link_id = "P103_gw180"
direction = "l"

"""
Try to run ech function at once and make sure each of them does what they are supposed to do to make sure you are putting the 
right inputs. Moreover, it is recommendable to save a copy in case something goes wrong as this will rewrite the whole file.
"""

In [24]:
silencing_links(path,x1,x2,y1,y2)

In [25]:
creating_abcr(path,plat_id,signal_id,direction)

In [26]:
creating_links(path,x1,x2,y1,y2,link_id,direction)